# Tweets streaming - Producer
Load Tweets, clean and send to localhost

In [0]:
# !pip install tweepy
# !pip install tweet-preprocessor

In [0]:
import tweepy 
from tweepy import OAuthHandler # to authenticate Twitter API
from tweepy import Stream 
from tweepy.streaming import StreamListener
import socket 
import json 
from datetime import datetime
import preprocessor as p

In [0]:
# My API credentials
consumer_key = '...'
consumer_secret = '...'
access_token = '...'
access_secret = '...'


In [0]:
# Preprocessing options used for p.clean() below
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)


In [0]:
# Define Listener and perform some basic preprocessing on the Tweet text
class TweetsListener(StreamListener):
    # initialized the constructor
    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            # read the Twitter data which comes as a JSON format 
            # select the actual tweet in 'text' of the JSON file and clean the text
            raw = json.loads(data)
            msg = p.clean(raw["text"])

            # we need to add a timestamp to the message and add a 'end of tweet' marker
            stamp = datetime.now().isoformat(timespec='milliseconds')
            print(str(msg + " stop " + stamp + "t_end").encode('utf-8'))

            # the actual tweet data is sent to the client socket (mind the '\n' at the end of the line!)
            self.client_socket.send(str(msg + " stop " + stamp + "t_end\n").encode('utf-8'))
            return True

        except BaseException as e:
            # Error handling
            print("This is wrong : %s" % str(e))
            return True

    def on_error(self, status):
        print(status)
        return True


In [0]:
def sendData(c_socket):
    # authentication
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # twitter_stream will get the actual live tweet data
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    # filter the tweet feeds related to "corona"
    twitter_stream.filter(track=['bitcoin'])
    # in case you want to pass multiple criteria
    # twitter_stream.filter(track=['DataScience','python','Iot'])


In [0]:
# create a socket object
s = socket.socket()

# Get local machine name : host and port
host = "localhost"
port = 9997

# Bind to the port
s.bind((host, port))
print("Listening on port: %s" % str(port))

# Wait and Establish the connection with client.
s.listen(5)
c, addr = s.accept()

print("Received request from: " + str(addr))

# Keep the stream data available
sendData(c)
